In [1]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI/Ad Classifier/Videos_data.csv')
data.head()

,Unnamed: 0,Video Id,Title,Description,Category
0,0,ehmsJLZlCZ0,Ep 1| Travelling through North East India | Of...,"The journey to Arunachal, North East India beg...",travel
1,1,e2NQE41J5eM,How do I travel so much ! How do I earn money!!,SUBSCRIBE - https://goo.gl/dEtSMJ ('MountainTr...,travel
2,2,i9E_Blai8vk,TRAVEL VLOG ∙ Welcome to Bali | PRISCILLA LEE,I had the chance to fly out to Bali with my wh...,travel
3,3,#NAME?,GOA TRAVEL DIARY | FOUR DAYS IN GOA | TRAVEL O...,Hope you enjoy MY GOA TRAVEL DIARY this video!...,travel
4,4,7ByoBJYXU0k,5 Steps to Becoming a Travel Blogger,"Travel blogger, Nikki Vargas, of The Pin the M...",travel


In [2]:
import nltk as nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'haven', 'mustn', 'they', 'yourselves', 'his', 'if', "shouldn't", 'an', 'do', 'while', "that'll", "it's", 'have', 'was', 'both', 'them', 'should', 'isn', 'shan', 'into', 'yourself', 'mightn', "wouldn't", 'any', 'doing', "doesn't", "aren't", 'ours', 'with', 'herself', 'why', 'above', 'or', 'own', 'and', 'myself', 'more', 'theirs', 'few', 'weren', 'there', 'himself', 'the', 't', 'did', "weren't", 'whom', 'same', 'before', 'can', "you're", 'didn', 'to', 'couldn', 'll', 'a', 'down', 'in', 'i', 'again', "mightn't", 'aren', 'ourselves', 's', 'be', 'been', "don't", 'of', 'o', 'further', "you'd", "hasn't", 'me', 'my', "haven't", 'her', 'she', 'how', 'not', 'am', 'nor', 'had', 'some', 'needn', 'having', 'does', 'than', 'hasn', 'below', 'won', 'through', 'about', 'being', 'for', 'too', 'then', 'ma', 'which', "couldn't", 'hers', 'each', 've', 'he', 'its', 'such', 'we', 'by', 'between', 'their', 'these', 'shouldn', 'is', 'wouldn', 'after', 'until', 'm', 'itself', 'hadn', 'you', 'yours', "shan't",

In [4]:
nltk.download('wordnet')
data = data.dropna()
def pre_process(text):

    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    ##Convert to list from string
    text = text.split()

    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word, "v") for word in text]

    return ' '.join(text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
import re
from nltk.stem import WordNetLemmatizer
data['Title'] = data['Title'].apply(lambda x:pre_process(x))
data['Description'] = data['Description'].apply(lambda x:pre_process(x))

In [6]:
data.head()

,Unnamed: 0,Video Id,Title,Description,Category
0,0,ehmsJLZlCZ0,travel north east india arunachal journey begi...,journey arunachal north east india begin train...,travel
1,1,e2NQE41J5eM,travel much earn money,subscribe https goo detsmj mountaintrekker gim...,travel
2,2,i9E_Blai8vk,travel vlog welcome bali priscilla lee,chance fly bali whole family thanksgiving firs...,travel
3,3,#NAME?,goa travel diary four days goa travel outfit i...,hope enjoy goa travel diary video forget subsc...,travel
4,4,7ByoBJYXU0k,step become travel blogger,travel blogger nikki vargas pin map project vo...,travel


In [7]:
data['Category'].value_counts()

,count
Category,
food,1695
manufacturing,1673
art and music,1672
travel,1671
science and technology,1656
history,1632


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Category'] = le.fit_transform(data['Category'])

In [9]:
data.head()

,Unnamed: 0,Video Id,Title,Description,Category
0,0,ehmsJLZlCZ0,travel north east india arunachal journey begi...,journey arunachal north east india begin train...,5
1,1,e2NQE41J5eM,travel much earn money,subscribe https goo detsmj mountaintrekker gim...,5
2,2,i9E_Blai8vk,travel vlog welcome bali priscilla lee,chance fly bali whole family thanksgiving firs...,5
3,3,#NAME?,goa travel diary four days goa travel outfit i...,hope enjoy goa travel diary video forget subsc...,5
4,4,7ByoBJYXU0k,step become travel blogger,travel blogger nikki vargas pin map project vo...,5


In [10]:
le.classes_

array(['art and music', 'food', 'history', 'manufacturing',
       'science and technology', 'travel'], dtype=object)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_title = TfidfVectorizer(sublinear_tf=True,max_features=1000,ngram_range=(1,2),stop_words='english')
tfidf_desc = TfidfVectorizer(sublinear_tf=True,max_features=1000,ngram_range=(1,2),stop_words='english')
labels = data['Category']
features_title = tfidf_title.fit_transform(data['Title']).toarray()
features_desc = tfidf_desc.fit_transform(data['Description']).toarray()

In [12]:
features_title.shape

(9999, 1000)

In [15]:
!pip install keras
import keras
from keras.preprocessing.text import Tokenizer # This should now work after installing keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 50
# This is fixed.
EMBEDDING_DIM = 100

# Combining titles and descriptions into a single sentence
titles = data['Title'].values
descriptions = data['Description'].values
data_for_lstms = []
for i in range(len(titles)):
    temp_list = [titles[i], descriptions[i]]
    data_for_lstms.append(' '.join(temp_list))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_for_lstms)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Convert the data to padded sequences
X = tokenizer.texts_to_sequences(data_for_lstms)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

# One-hot Encode labels
Y = pd.get_dummies(data['classes']).values
print('Shape of label tensor:', Y.shape)

# Splitting into training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 42)

ModuleNotFoundError: No module named 'keras.preprocessing.text'